# convblock

In [1]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [3]:

def init_filter(d, mi, mo, stride):
  return (np.random.randn(d, d, mi, mo) * np.sqrt(2.0 / (d * d * mi))).astype(np.float32)


In [2]:
class ConvLayer:
  def __init__(self, d, mi, mo, stride=2, padding='VALID'):
    self.W = tf.Variable(init_filter(d, mi, mo, stride))
    self.b = tf.Variable(np.zeros(mo, dtype=np.float32))
    self.stride = stride
    self.padding = padding

  def forward(self, X):
    X = tf.nn.conv2d(
      X,
      self.W,
      strides=[1, self.stride, self.stride, 1],
      padding=self.padding
    )
    X = X + self.b
    return X

  def copyFromKerasLayers(self, layer):
    # only 1 layer to copy from
    W, b = layer.get_weights()
    op1 = self.W.assign(W)
    op2 = self.b.assign(b)
    self.session.run((op1, op2))

  # def copyFromWeights(self, W, b):
  #   op1 = self.W.assign(W)
  #   op2 = self.b.assign(b)
  #   self.session.run((op1, op2))

  def get_params(self):
    return [self.W, self.b]



In [4]:
class BatchNormLayer:
  def __init__(self, D):
    self.running_mean = tf.Variable(np.zeros(D, dtype=np.float32), trainable=False)
    self.running_var  = tf.Variable(np.ones(D, dtype=np.float32), trainable=False)
    self.gamma        = tf.Variable(np.ones(D, dtype=np.float32))
    self.beta         = tf.Variable(np.zeros(D, dtype=np.float32))

  def forward(self, X):
    return tf.nn.batch_normalization(
      X,
      self.running_mean,
      self.running_var,
      self.beta,
      self.gamma,
      1e-3
    )

  def copyFromKerasLayers(self, layer):
    # only 1 layer to copy from
    # order:
    # gamma, beta, moving mean, moving variance
    gamma, beta, running_mean, running_var = layer.get_weights()
    op1 = self.running_mean.assign(running_mean)
    op2 = self.running_var.assign(running_var)
    op3 = self.gamma.assign(gamma)
    op4 = self.beta.assign(beta)
    self.session.run((op1, op2, op3, op4))

  def get_params(self):
    return [self.running_mean, self.running_var, self.gamma, self.beta]


In [5]:

class ConvBlock:
  def __init__(self, mi, fm_sizes, stride=2, activation=tf.nn.relu):
    # conv1, conv2, conv3
    # note: # feature maps shortcut = # feauture maps conv 3
    assert(len(fm_sizes) == 3)

    # note: kernel size in 2nd conv is always 3
    #       so we won't bother including it as an arg

    # note: stride only applies to conv 1 in main branch
    #       and conv in shortcut, otherwise stride is 1

    self.session = None
    self.f = tf.nn.relu
    
    # init main branch
    # Conv -> BN -> F() ---> Conv -> BN -> F() ---> Conv -> BN
    self.conv1 = ConvLayer(1, mi, fm_sizes[0], stride)
    self.bn1   = BatchNormLayer(fm_sizes[0])
    self.conv2 = ConvLayer(3, fm_sizes[0], fm_sizes[1], 1, 'SAME')
    self.bn2   = BatchNormLayer(fm_sizes[1])
    self.conv3 = ConvLayer(1, fm_sizes[1], fm_sizes[2], 1)
    self.bn3   = BatchNormLayer(fm_sizes[2])

    # init shortcut branch
    # Conv -> BN
    self.convs = ConvLayer(1, mi, fm_sizes[2], stride)
    self.bns   = BatchNormLayer(fm_sizes[2])

    # in case needed later
    self.layers = [
      self.conv1, self.bn1,
      self.conv2, self.bn2,
      self.conv3, self.bn3,
      self.convs, self.bns
    ]

    # this will not be used when input passed in from
    # a previous layer
    self.input_ = tf.placeholder(tf.float32, shape=(1, 224, 224, mi))
    self.output = self.forward(self.input_)

  def forward(self, X):
    # main branch
    FX = self.conv1.forward(X)
    FX = self.bn1.forward(FX)
    FX = self.f(FX)
    FX = self.conv2.forward(FX)
    FX = self.bn2.forward(FX)
    FX = self.f(FX)
    FX = self.conv3.forward(FX)
    FX = self.bn3.forward(FX)

    # shortcut branch
    SX = self.convs.forward(X)
    SX = self.bns.forward(SX)

    return self.f(FX + SX)

  def predict(self, X):
    assert(self.session is not None)
    return self.session.run(
      self.output,
      feed_dict={self.input_: X}
    )

  def set_session(self, session):
    # need to make this a session
    # so assignment happens on sublayers too
    self.session = session
    self.conv1.session = session
    self.bn1.session = session
    self.conv2.session = session
    self.bn2.session = session
    self.conv3.session = session
    self.bn3.session = session
    self.convs.session = session
    self.bns.session = session

  def copyFromKerasLayers(self, layers):
    # [<keras.layers.convolutional.Conv2D at 0x117bd1978>,
    #  <keras.layers.normalization.BatchNormalization at 0x117bf84a8>,
    #  <keras.layers.core.Activation at 0x117c15fd0>,
    #  <keras.layers.convolutional.Conv2D at 0x117c23be0>,
    #  <keras.layers.normalization.BatchNormalization at 0x117c51978>,
    #  <keras.layers.core.Activation at 0x117c93518>,
    #  <keras.layers.convolutional.Conv2D at 0x117cc1518>,
    #  <keras.layers.convolutional.Conv2D at 0x117d21630>,
    #  <keras.layers.normalization.BatchNormalization at 0x117cd2a58>,
    #  <keras.layers.normalization.BatchNormalization at 0x117d44b00>,
    #  <keras.layers.merge.Add at 0x117dae748>,
    #  <keras.layers.core.Activation at 0x117da2eb8>]
    self.conv1.copyFromKerasLayers(layers[0])
    self.bn1.copyFromKerasLayers(layers[1])
    self.conv2.copyFromKerasLayers(layers[3])
    self.bn2.copyFromKerasLayers(layers[4])
    self.conv3.copyFromKerasLayers(layers[6])
    self.bn3.copyFromKerasLayers(layers[8])
    self.convs.copyFromKerasLayers(layers[7])
    self.bns.copyFromKerasLayers(layers[9])

  def get_params(self):
    params = []
    for layer in self.layers:
      params += layer.get_params()
    return params


In [6]:
if __name__ == '__main__':
  conv_block = ConvBlock(mi=3, fm_sizes=[64, 64, 256], stride=1)

  # make a fake image
  X = np.random.random((1, 224, 224, 3))

  init = tf.global_variables_initializer()
  with tf.Session() as session:
    conv_block.set_session(session)
    session.run(init)

    output = conv_block.predict(X)
    print("output.shape:", output.shape)


output.shape: (1, 224, 224, 256)


In [7]:
output

array([[[[9.98377323e-01, 1.18946671e+00, 6.80550575e-01, ...,
          0.00000000e+00, 0.00000000e+00, 1.74632192e+00],
         [1.71091628e+00, 1.18397129e+00, 1.86851668e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [6.18836582e-01, 3.99079740e-01, 6.40911937e-01, ...,
          4.06729460e-01, 0.00000000e+00, 0.00000000e+00],
         ...,
         [1.47901881e+00, 1.27081215e+00, 1.80087328e+00, ...,
          0.00000000e+00, 0.00000000e+00, 6.24309063e-01],
         [7.42667079e-01, 2.63923526e-01, 8.39986444e-01, ...,
          2.42179096e-01, 0.00000000e+00, 0.00000000e+00],
         [4.36642841e-02, 5.63806593e-02, 3.51840397e-03, ...,
          4.41147029e-01, 3.50591421e-01, 0.00000000e+00]],

        [[2.87527514e+00, 3.90360773e-01, 1.89426565e+00, ...,
          2.93770432e-01, 0.00000000e+00, 0.00000000e+00],
         [2.54572940e+00, 6.85984492e-01, 2.14225197e+00, ...,
          0.00000000e+00, 4.43948627e-01, 0.00000000e+00],
        

# Identify Block

In [8]:

class IdentityBlock:
  def __init__(self, mi, fm_sizes, activation=tf.nn.relu):
    # conv1, conv2, conv3
    # note: # feature maps shortcut = # feauture maps conv 3
    assert(len(fm_sizes) == 3)

    # note: kernel size in 2nd conv is always 3
    #       so we won't bother including it as an arg

    self.session = None
    self.f = tf.nn.relu
    
    # init main branch
    # Conv -> BN -> F() ---> Conv -> BN -> F() ---> Conv -> BN
    self.conv1 = ConvLayer(1, mi, fm_sizes[0], 1)
    self.bn1   = BatchNormLayer(fm_sizes[0])
    self.conv2 = ConvLayer(3, fm_sizes[0], fm_sizes[1], 1, 'SAME')
    self.bn2   = BatchNormLayer(fm_sizes[1])
    self.conv3 = ConvLayer(1, fm_sizes[1], fm_sizes[2], 1)
    self.bn3   = BatchNormLayer(fm_sizes[2])

    # in case needed later
    self.layers = [
      self.conv1, self.bn1,
      self.conv2, self.bn2,
      self.conv3, self.bn3,
    ]

    # this will not be used when input passed in from
    # a previous layer
    self.input_ = tf.placeholder(tf.float32, shape=(1, 224, 224, mi))
    self.output = self.forward(self.input_)

  def forward(self, X):
    # main branch
    FX = self.conv1.forward(X)
    FX = self.bn1.forward(FX)
    FX = self.f(FX)
    FX = self.conv2.forward(FX)
    FX = self.bn2.forward(FX)
    FX = self.f(FX)
    FX = self.conv3.forward(FX)
    FX = self.bn3.forward(FX)

    return self.f(FX + X)

  def predict(self, X):
    assert(self.session is not None)
    return self.session.run(
      self.output,
      feed_dict={self.input_: X}
    )

  def set_session(self, session):
    # need to make this a session
    # so assignment happens on sublayers too
    self.session = session
    self.conv1.session = session
    self.bn1.session = session
    self.conv2.session = session
    self.bn2.session = session
    self.conv3.session = session
    self.bn3.session = session

  def copyFromKerasLayers(self, layers):
    assert(len(layers) == 10)
    # <keras.layers.convolutional.Conv2D at 0x7fa44255ff28>,
    # <keras.layers.normalization.BatchNormalization at 0x7fa44250e7b8>,
    # <keras.layers.core.Activation at 0x7fa44252d9e8>,
    # <keras.layers.convolutional.Conv2D at 0x7fa44253af60>,
    # <keras.layers.normalization.BatchNormalization at 0x7fa4424e4f60>,
    # <keras.layers.core.Activation at 0x7fa442494828>,
    # <keras.layers.convolutional.Conv2D at 0x7fa4424a2da0>,
    # <keras.layers.normalization.BatchNormalization at 0x7fa44244eda0>,
    # <keras.layers.merge.Add at 0x7fa44245d5c0>,
    # <keras.layers.core.Activation at 0x7fa44240aba8>
    self.conv1.copyFromKerasLayers(layers[0])
    self.bn1.copyFromKerasLayers(layers[1])
    self.conv2.copyFromKerasLayers(layers[3])
    self.bn2.copyFromKerasLayers(layers[4])
    self.conv3.copyFromKerasLayers(layers[6])
    self.bn3.copyFromKerasLayers(layers[7])

  def get_params(self):
    params = []
    for layer in self.layers:
      params += layer.get_params()
    return params




In [9]:
if __name__ == '__main__':
  identity_block = IdentityBlock(mi=256, fm_sizes=[64, 64, 256])

  # make a fake image
  X = np.random.random((1, 224, 224, 256))

  init = tf.global_variables_initializer()
  with tf.Session() as session:
    identity_block.set_session(session)
    session.run(init)

    output = identity_block.predict(X)
    print("output.shape:", output.shape)


output.shape: (1, 224, 224, 256)


# 50 layers ResNet Impl

![](https://postfiles.pstatic.net/20160718_2/laonple_1468811199171uLSM4_PNG/%C0%CC%B9%CC%C1%F6_80.png?type=w2)

In [ ]:
class TFResNet:
  def __init__(self):
    self.layers = [
      # before conv block
      ConvLayer(d=7, mi=3, mo=64, stride=2, padding='SAME'),
      BatchNormLayer(64),
      ReLULayer(),
      MaxPoolLayer(dim=3),
      # conv block
      ConvBlock(mi=64, fm_sizes=[64, 64, 256], stride=1),
      # identity block x 2
      IdentityBlock(mi=256, fm_sizes=[64, 64, 256]),
      IdentityBlock(mi=256, fm_sizes=[64, 64, 256]),
      # conv block
      ConvBlock(mi=256, fm_sizes=[128, 128, 512], stride=2),
      # identity block x 3
      IdentityBlock(mi=512, fm_sizes=[128, 128, 512]),
      IdentityBlock(mi=512, fm_sizes=[128, 128, 512]),
      IdentityBlock(mi=512, fm_sizes=[128, 128, 512]),
      # conv block
      ConvBlock(mi=512, fm_sizes=[256, 256, 1024], stride=2),
      # identity block x 5
      IdentityBlock(mi=1024, fm_sizes=[256, 256, 1024]),
      IdentityBlock(mi=1024, fm_sizes=[256, 256, 1024]),
      IdentityBlock(mi=1024, fm_sizes=[256, 256, 1024]),
      IdentityBlock(mi=1024, fm_sizes=[256, 256, 1024]),
      IdentityBlock(mi=1024, fm_sizes=[256, 256, 1024]),
      # conv block
      ConvBlock(mi=1024, fm_sizes=[512, 512, 2048], stride=2),
      # identity block x 2
      IdentityBlock(mi=2048, fm_sizes=[512, 512, 2048]),
      IdentityBlock(mi=2048, fm_sizes=[512, 512, 2048]),
      # pool / flatten / dense
      AvgPool(ksize=7),
      Flatten(),
      DenseLayer(mi=2048, mo=1000)
    ]
    self.input_ = tf.placeholder(tf.float32, shape=(None, 224, 224, 3))
    self.output = self.forward(self.input_)



# ResNet Transfer learning


In [ ]:

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50, preprocess_input
# from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from glob import glob


# re-size all the images to this
IMAGE_SIZE = [224, 224] # feel free to change depending on dataset

# training config:
epochs = 16
batch_size = 32

# https://www.kaggle.com/paultimothymooney/blood-cells
train_path = '../large_files/blood_cell_images/TRAIN'
valid_path = '../large_files/blood_cell_images/TEST'

# https://www.kaggle.com/moltean/fruits
# train_path = '../large_files/fruits-360/Training'
# valid_path = '../large_files/fruits-360/Validation'
# train_path = '../large_files/fruits-360-small/Training'
# valid_path = '../large_files/fruits-360-small/Validation'

# useful for getting number of files
image_files = glob(train_path + '/*/*.jp*g')
valid_image_files = glob(valid_path + '/*/*.jp*g')

# useful for getting number of classes
folders = glob(train_path + '/*')


# look at an image for fun
plt.imshow(image.load_img(np.random.choice(image_files)))
plt.show()


# add preprocessing layer to the front of VGG
res = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in res.layers:
  layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(res.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)


# create a model object
model = Model(inputs=res.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)



# create an instance of ImageDataGenerator
gen = ImageDataGenerator(
  rotation_range=20,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.2,
  horizontal_flip=True,
  vertical_flip=True,
  preprocessing_function=preprocess_input
)


# test generator to see how it works and some other useful things

# get label mapping for confusion matrix plot later
test_gen = gen.flow_from_directory(valid_path, target_size=IMAGE_SIZE)
print(test_gen.class_indices)
labels = [None] * len(test_gen.class_indices)
for k, v in test_gen.class_indices.items():
  labels[v] = k

# should be a strangely colored image (due to VGG weights being BGR)
for x, y in test_gen:
  print("min:", x[0].min(), "max:", x[0].max())
  plt.title(labels[np.argmax(y[0])])
  plt.imshow(x[0])
  plt.show()
  break


# create generators
train_generator = gen.flow_from_directory(
  train_path,
  target_size=IMAGE_SIZE,
  shuffle=True,
  batch_size=batch_size,
)
valid_generator = gen.flow_from_directory(
  valid_path,
  target_size=IMAGE_SIZE,
  shuffle=True,
  batch_size=batch_size,
)


# fit the model
r = model.fit_generator(
  train_generator,
  validation_data=valid_generator,
  epochs=epochs,
  steps_per_epoch=len(image_files) // batch_size,
  validation_steps=len(valid_image_files) // batch_size,
)



def get_confusion_matrix(data_path, N):
  # we need to see the data in the same order
  # for both predictions and targets
  print("Generating confusion matrix", N)
  predictions = []
  targets = []
  i = 0
  for x, y in gen.flow_from_directory(data_path, target_size=IMAGE_SIZE, shuffle=False, batch_size=batch_size * 2):
    i += 1
    if i % 50 == 0:
      print(i)
    p = model.predict(x)
    p = np.argmax(p, axis=1)
    y = np.argmax(y, axis=1)
    predictions = np.concatenate((predictions, p))
    targets = np.concatenate((targets, y))
    if len(targets) >= N:
      break

  cm = confusion_matrix(targets, predictions)
  return cm


cm = get_confusion_matrix(train_path, len(image_files))
print(cm)
valid_cm = get_confusion_matrix(valid_path, len(valid_image_files))
print(valid_cm)


# plot some data

# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()

from util import plot_confusion_matrix
plot_confusion_matrix(cm, labels, title='Train confusion matrix')
plot_confusion_matrix(valid_cm, labels, title='Validation confusion matrix')